# JustWalk Steps

In [2]:
from pybehavior.models import DataSet
from pybehavior.tools import Loader, Preprocessor
import pandas as pd
import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

In [ ]:
data = pd.read_pickle('data/walk/data7.pickle')
data.tolist()


# JustWalk EMA

In [36]:

data = pd.read_pickle('data/ema/data3.pickle')

data = data.groupby(['rnum', 'created', 'question_id']).agg({"answer_value": "mean"})
data = data.pivot_table(index=['rnum', 'created'], columns=['question_id'], values='answer_value')
data = data.sort_values(['rnum', 'created']).reset_index()
data.columns.name = None

data

,rnum,created,E10,E11,E12.1.1,E12.1.2,E12.4,E12.7,E13.1,E13.2,E13.3,E13.4,E13.5,E13.6,E13.7,E14,E5,E6,E7,E8
0,1,2022-06-30,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,2.0,1.0
1,1,2022-07-02,2.0,2.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,2.0,3.0
2,1,2022-07-03,0.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,4.0
3,1,2022-07-04,2.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
4,1,2022-07-09,2.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,48,2022-09-14,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,1.0,4.0
1576,48,2022-09-16,3.0,3.0,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3.0,3.0,3.0,1.0
1577,48,2022-09-18,1.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,3.0,3.0
1578,48,2022-09-19,0.0,2.0,0.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.0


In [37]:
data = pd.read_pickle('data/ema/data4.pickle')
data

question_id,rnum,created,E10,E11,E12.1.1,E12.1.2,E12.4,E12.7,E13.1,E13.2,E13.3,E13.4,E13.5,E13.6,E13.7,E14,E5,E6,E7,E8
0,1,2022-06-30,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,2.0,1.0
1,1,2022-07-02,2.0,2.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,2.0,3.0
2,1,2022-07-03,0.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,4.0
3,1,2022-07-04,2.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
4,1,2022-07-09,2.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,48,2022-09-14,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,1.0,4.0
1576,48,2022-09-16,3.0,3.0,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3.0,3.0,3.0,1.0
1577,48,2022-09-18,1.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,3.0,3.0
1578,48,2022-09-19,0.0,2.0,0.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.0


In [4]:
date_span = data.groupby('rnum').agg({'created': ['min', 'max']})
print(date_span)

question_id     created            
                    min         max
rnum                               
1            2022-06-30  2022-09-21
2            2022-07-21  2022-08-06
3            2022-06-26  2022-08-12
4            2022-06-20  2022-09-21
5            2022-07-03  2022-09-21
6            2022-06-30  2022-09-21
7            2022-07-13  2022-08-30
8            2022-07-02  2022-09-22
9            2022-06-27  2022-09-21
10           2022-06-30  2022-09-19
11           2022-06-28  2022-09-21
13           2022-06-27  2022-07-05
14           2022-08-06  2022-09-02
15           2022-07-07  2022-09-12
16           2022-07-06  2022-08-16
17           2022-07-03  2022-09-05
18           2022-06-26  2022-09-20
19           2022-06-13  2022-08-31
22           2022-06-26  2022-09-21
24           2022-07-19  2022-08-05
25           2022-07-05  2022-09-21
26           2022-06-26  2022-09-21
27           2022-06-29  2022-09-20
28           2022-07-02  2022-09-13
29           2022-07-16  202

In [ ]:
data = pd.read_pickle('data/ema/data3.pickle')

display(data.shape)
display(data.drop_duplicates().shape)
display(data[['rnum', 'created']].drop_duplicates().shape)
display(data[['rnum', 'created', 'question_id']].drop_duplicates().shape)




In [ ]:
data = pd.read_pickle('data/ema/data4.pickle')
data.to_csv('justwalk.ema.20220920.csv')

In [ ]:
data = data.pivot_table(index=['rnum', 'created'], columns='question_id', values='answer_value')
data.to_csv('data.csv')